In [3]:
#import
import os
import sys

import tqdm
import math
import torch
import logging

import numpy as np
import pandas as pd
import torch.nn as nn
import lightning.pytorch as pl
import matplotlib.pyplot as plt

from scipy import spatial
from scipy.stats import chisquare, kstest
from scipy.optimize import curve_fit
from torchmetrics import MeanAbsoluteError
from sklearn.preprocessing import StandardScaler
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from torch.optim.lr_scheduler import ReduceLROnPlateau
from lightning.pytorch.callbacks.early_stopping import EarlyStopping
from lightning.pytorch.callbacks import Callback, LearningRateMonitor, ModelCheckpoint

In [4]:
data_path = './data/clasdb_pi_plus_n.txt'
df = pd.read_csv(data_path, delimiter='\t', header=None)
df.columns = ['Ebeam', 'W', 'Q2', 'cos_theta', 'phi', 'dsigma_dOmega', 'error', 'id']
df.loc[8314:65671, 'Ebeam'] = 5.754  # peculiarity of this dataset.
df = df[~((df.Ebeam == 5.754) & (~df.Q2.isin([1.715, 2.050, 2.445, 2.915, 3.480, 4.155])))] # peculiarity of this dataset #2
df['phi'] = df.phi.apply(lambda x: math.radians(x))
df['weight'] = df['error'].apply(lambda x: x and 1 / x or 100)  # x and 1 / x or 100  is just a reversed error but with validation 1/0 error in this case it will return 100
df = df.drop('id', axis=1)
df = df.drop_duplicates(subset=['Ebeam', 'W', 'Q2', 'cos_theta', 'phi'])

In [10]:
def func_cos(x, a, b, c):
        return a + b*np.cos(2*x) + c*np.cos(x)

def fit_to_func(x):
    try:
        popt, pcov = curve_fit(func_cos, x.phi, x.dsigma_dOmega, sigma=x.error, absolute_sigma=True)
        a, b, c = popt[0], popt[1], popt[2]
        
        a_error, b_error, c_error = np.sqrt(pcov[0][0]), np.sqrt(pcov[1][1]), np.sqrt(pcov[2][2])
        
        return a, b, c, a_error, b_error, c_error
    except Exception as e:
        return None, None, None, None, None, None
    
df_sf = df.groupby(['Ebeam', 'W', 'Q2', 'cos_theta']).apply(lambda x: fit_to_func(x)).reset_index()

df_sf['A'] = df_sf[0].apply(lambda x: x[0])
df_sf['B'] = df_sf[0].apply(lambda x: x[1])
df_sf['C'] = df_sf[0].apply(lambda x: x[2])

df_sf['A_error'] = df_sf[0].apply(lambda x: x[3])
df_sf['B_error'] = df_sf[0].apply(lambda x: x[4])
df_sf['C_error'] = df_sf[0].apply(lambda x: x[5])

df_sf.drop(0, axis=1, inplace=True)
df_sf.dropna(inplace=True)

In [12]:
df

,Ebeam,W,Q2,cos_theta,phi,dsigma_dOmega,error,weight
0,1.515,1.11,0.3,0.991445,0.261799,15.3700,5.264366,0.189956
1,1.515,1.11,0.3,0.991445,0.785398,4.5110,1.743136,0.573679
2,1.515,1.11,0.3,0.991445,1.308997,4.4780,1.611260,0.620632
3,1.515,1.11,0.3,0.991445,1.832596,5.1360,1.523529,0.656371
4,1.515,1.11,0.3,0.991445,2.356194,5.0780,1.219442,0.820047
...,...,...,...,...,...,...,...,...
98022,5.499,2.01,4.0,0.975000,3.730641,0.1012,0.043165,23.166714
98023,5.499,2.01,4.0,0.975000,3.992441,0.1199,0.076638,13.048361
98024,5.499,2.01,4.0,0.975000,4.646939,0.1578,0.095391,10.483159
98025,5.499,2.01,4.0,0.975000,4.777839,0.2346,0.158557,6.306878


In [1]:
from torchmetrics import MeanAbsoluteError, MeanSquaredError, RootMeanSquaredErrorUsingSlidingWindow
